Multilayer Perceptron to Predict International Airline Passengers (t+1, given t, t-1, t-2). In here we use a simple deep learning fundamantal model to predic airline passenger for last century.

Firstly import various library which are used here.

In [ ]:
import numpy
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams["figure.figsize"] = (18,9)
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense

Convert an array of values into a dataset matrix

In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

Load the dataset. Here dataset contain ten(1949-1960) years monthly data.

In [ ]:
dataframe = read_csv("../input/international-airline-passengers/international-airline-passengers.csv", usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

Split into train and test sets

In [ ]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

Reshape dataset

In [ ]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

Create and fit Multilayer Perceptron model

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=look_back, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=300, batch_size=2, verbose=2)

Estimate model performance

In [ ]:
trainScore = model.evaluate(trainX, trainY, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(testX, testY, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

Generate predictions for training

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

Shift train predictions for plotting

In [ ]:
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

Shift test predictions for plotting

In [ ]:
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

Plot baseline and predictions

In [ ]:
plt.plot(dataset,label='Data')
plt.plot(trainPredictPlot,label='Train Prediction')
plt.plot(testPredictPlot,label='Test Prediction')
plt.xlabel ('Months',fontsize=20)
plt.ylabel ('Passengers Numbers',fontsize=20)
plt.legend(fontsize=20)
plt.show()